QUESTION 3

************************************** part a ******************************************

The NAICS or North American Industry Classification System, as its name suggetsts, is a system to classify business establishments by the type of economics activity or process of production. It uses numerical code to classify one type of business from another. The first two digits classify sector, the third digit classifies subsector (unique to that sector), the fourth industry group, and the fifth industry. Hence the higher the digit position, the more specific the classification becomes. 

i) 31111 is dog, cat, and other animal food manufacturing

ii) 31121 is flour, rice, and malt manufacturing

iii) 31131 is beet sugar and cane sugar manufacturing

In [67]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf 
import statsmodels.api as sm
import linearmodels as lm

************************************** part b ******************************************

In [68]:
dfs = pd.read_stata(r'C:\Users\devin\Downloads\estdata.dta')
dfs

,year,naics5_02g,emp,shipments,nplants,nfirms,mval,xval
0,1997.0,31111,46651.0,2.773235e+10,1696.0,1077.0,3.882001e+08,1.316385e+09
1,2002.0,31111,47080.0,2.802495e+10,1811.0,1211.0,4.082143e+08,1.373442e+09
2,2007.0,31111,46227.0,3.917388e+10,1767.0,1192.0,6.770921e+08,1.992830e+09
3,2012.0,31111,44873.0,5.790565e+10,1690.0,1109.0,1.265167e+09,2.750734e+09
4,1997.0,31121,17877.0,1.114188e+10,483.0,324.0,3.219493e+08,1.166381e+09
...,...,...,...,...,...,...,...,...
665,2012.0,33995,68384.0,1.074047e+10,5454.0,5357.0,2.240829e+08,1.241907e+08
666,1997.0,33999,157947.0,1.880817e+10,4708.0,4475.0,6.521592e+09,2.957116e+09
667,2002.0,33999,185411.0,2.509032e+10,7174.0,6960.0,9.226218e+09,3.405874e+09
668,2007.0,33999,145633.0,2.841949e+10,7768.0,7569.0,1.195575e+10,5.359205e+09


In [69]:
total_absorption = dfs['shipments'] + dfs['mval'] - dfs['xval']

In [70]:
delta_abs = []
delta_pct_nfirms = []

for i in dfs.index[dfs['year'] != 1997]: 
    delta_abs.append((dfs.mval[i] - dfs.mval[i-1])/total_absorption[i-1])
    delta_pct_nfirms.append((dfs.nfirms[i] - dfs.nfirms[i-1])/dfs.nfirms[i-1])

In [21]:
#Creating year dummies for Fixed-Effects regression

newyear = list(dfs.year[dfs.year != 1997])
newyear_str = [str(x) for x in newyear]
year_dummies = pd.get_dummies(newyear_str, drop_first=True)

#Creating list of NAICS code for data from 02 to 12 for the new shortened data frame below

naics5_02g = list(dfs.naics5_02g[dfs.year != 1997])

#Creating dfs2 (values for 02 to 12 only)

dfs2 = pd.DataFrame(naics5_02g, columns=['naics5_02g'])
dfs2['year'] = newyear
dfs2['delta_pct_nfirms'] = delta_pct_nfirms
dfs2['delta_abs'] = delta_abs

#Alternatively, pd.concat([dfs2, pd.Series(delta_abs)], axis=1)

dfs2 = pd.concat([dfs2, year_dummies], axis=1)
dfs2

,naics5_02g,year,delta_pct_nfirms,delta_abs,2007.0,2012.0
0,31111,2002.0,0.124420,0.000747,0,0
1,31111,2007.0,-0.015690,0.009936,1,0
2,31111,2012.0,-0.069631,0.015534,0,1
3,31121,2002.0,-0.120370,0.005877,0,0
4,31121,2007.0,-0.133333,0.045572,1,0
...,...,...,...,...,...,...
496,33995,2007.0,0.026269,0.009635,1,0
497,33995,2012.0,-0.148331,0.001139,0,1
498,33999,2002.0,0.555307,0.120890,0,0
499,33999,2007.0,0.087500,0.088304,1,0


In [71]:
#Creating the weights as 1/shipments

weights=[]
for i in dfs2.index:
    weights.append(dfs.shipments[i] ** -1)

mod_wls = sm.WLS(dfs2.iloc[:,2], dfs2.iloc[:,3:6], weights) 
mod_res = mod_wls.fit() 
print(mod_res.summary())

                                 WLS Regression Results                                
Dep. Variable:       delta_pct_nfirms   R-squared (uncentered):                   0.149
Model:                            WLS   Adj. R-squared (uncentered):              0.144
Method:                 Least Squares   F-statistic:                              29.05
Date:                Sat, 12 Mar 2022   Prob (F-statistic):                    2.53e-17
Time:                        04:50:55   Log-Likelihood:                         -35.804
No. Observations:                 501   AIC:                                      77.61
Df Residuals:                     498   BIC:                                      90.26
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

************************************** part c ******************************************

By including the time fixed effects, we observe that delta_abs becomes irrelevant. We also tried performing regressions without the time fixed effects in whcih we found delta_abs to be relevant. Our finding is not consistent with the Melitz model which suggests that number of firm decreases as trade opens up (imports increase). A reason for this could be tha firms are equally productive in the US and the effect of trade libralization to all frims are nearly identical, resulting in no winners or exiters. 

************************************** part d ******************************************

In [72]:
dfs3 = pd.read_stata(r'C:\Users\devin\Downloads\wBt_dot.dta')
dfs3

,year,naics5_02g,wBt_dot,share_B
0,2002.0,31111,-0.161628,0.938545
1,2007.0,31111,-1.135505,0.973470
2,2012.0,31111,0.007852,0.966897
3,2002.0,31121,-0.143165,0.664431
4,2007.0,31121,-0.107913,0.591924
...,...,...,...,...
462,2007.0,33995,1.149367,0.938940
463,2012.0,33995,0.255997,0.969495
464,2002.0,33999,0.206999,0.908459
465,2007.0,33999,0.204853,0.939872


In [73]:
#Despite the difference in # of rows, we merge the main dfs and the new dfs3 
#such that we don't lose any observations from the main dfs

dfs_wBt = pd.merge(dfs, dfs3, how='left', left_on = ['year', 'naics5_02g'], right_on = ['year','naics5_02g'])
dfs_wBt

,year,naics5_02g,emp,shipments,nplants,nfirms,mval,xval,wBt_dot,share_B
0,1997.0,31111,46651.0,2.773235e+10,1696.0,1077.0,3.882001e+08,1.316385e+09,NaN,NaN
1,2002.0,31111,47080.0,2.802495e+10,1811.0,1211.0,4.082143e+08,1.373442e+09,-0.161628,0.938545
2,2007.0,31111,46227.0,3.917388e+10,1767.0,1192.0,6.770921e+08,1.992830e+09,-1.135505,0.973470
3,2012.0,31111,44873.0,5.790565e+10,1690.0,1109.0,1.265167e+09,2.750734e+09,0.007852,0.966897
4,1997.0,31121,17877.0,1.114188e+10,483.0,324.0,3.219493e+08,1.166381e+09,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
665,2012.0,33995,68384.0,1.074047e+10,5454.0,5357.0,2.240829e+08,1.241907e+08,0.255997,0.969495
666,1997.0,33999,157947.0,1.880817e+10,4708.0,4475.0,6.521592e+09,2.957116e+09,NaN,NaN
667,2002.0,33999,185411.0,2.509032e+10,7174.0,6960.0,9.226218e+09,3.405874e+09,0.206999,0.908459
668,2007.0,33999,145633.0,2.841949e+10,7768.0,7569.0,1.195575e+10,5.359205e+09,0.204853,0.939872


In [75]:
delta_iv_abs = [np.nan]
wBt_not_nan = dfs_wBt.index[-np.isnan(dfs_wBt.wBt_dot)]

for i in range(1,670) :
    delta_iv_abs.append(dfs_wBt.wBt_dot[i] * dfs_wBt.mval[i-1] / total_absorption[i-1])
    
dfs_wBt['delta_iv_abs'] = delta_iv_abs

In [76]:
#Now we merge the delta_iv_abs into the shortened (regression ready) dataset dfs2
#Here we drop the variables for year=1997

dfs4 = pd.merge(dfs2, dfs_wBt.iloc[:, np.r_[0,1,10]])

#Wherever delta_abs is 0, delta_iv_abs in nan. Therefore, 

dfs4['delta_iv_abs'] = dfs4['delta_iv_abs'].fillna(0)

#adding the weights into the df because the ivreg function won't take lists

dfs4['weights'] = weights
dfs4

,naics5_02g,year,delta_pct_nfirms,delta_abs,2007.0,2012.0,delta_iv_abs,weights
0,31111,2002.0,0.124420,0.000747,0,0,-0.002341,3.605897e-11
1,31111,2007.0,-0.015690,0.009936,1,0,-0.017130,3.568249e-11
2,31111,2012.0,-0.069631,0.015534,0,1,0.000140,2.552721e-11
3,31121,2002.0,-0.120370,0.005877,0,0,-0.004476,1.726947e-11
4,31121,2007.0,-0.133333,0.045572,1,0,-0.004853,8.975149e-11
...,...,...,...,...,...,...,...,...
496,33995,2007.0,0.026269,0.009635,1,0,0.013114,3.451839e-11
497,33995,2012.0,-0.148331,0.001139,0,1,0.004082,3.392602e-11
498,33999,2002.0,0.555307,0.120890,0,0,0.060340,3.277736e-11
499,33999,2007.0,0.087500,0.088304,1,0,0.061144,2.599856e-11


In [77]:
#Performing the iv FE regression with the same weights as in b)

iv_mod = lm.iv.model.IV2SLS(dfs4.iloc[:, 2], dfs4.iloc[:, 4:6], dfs4.iloc[:, 3], dfs4.iloc[:, 6], weights = dfs4.iloc[:, 7])
print(iv_mod.fit())

                          IV-2SLS Estimation Summary                          
Dep. Variable:       delta_pct_nfirms   R-squared:                      0.0479
Estimator:                    IV-2SLS   Adj. R-squared:                 0.0421
No. Observations:                 501   F-statistic:                    57.550
Date:                Sat, Mar 12 2022   P-value (F-stat)                0.0000
Time:                        04:53:08   Distribution:                  chi2(3)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
2007.0        -0.0169     0.0414    -0.4089     0.6826     -0.0980      0.0642
2012.0        -0.0850     0.0243    -3.5016     0.00

************************************** part e ******************************************

Now delta_abs has a negative value and is significant, which is in line with the results of the Melitz Model. 